<a href="https://colab.research.google.com/github/beneDesp/GLDM3002/blob/finalProject/finalProjectTeam2_MW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part I


### 1. Design your CRISP Model as shown in Figure 1:

### 2.  Split the data into training (50%), validation (30%), and test (20%)datasets.

In [5]:
#run cell and use "Choose Files" GUI to load ToyotaCorolla.xlsx
from google.colab import files
uploaded = files.upload()

Saving ToyotaCorolla.xlsx to ToyotaCorolla (1).xlsx


In [6]:
import pandas as pd

corolla = pd.read_excel("ToyotaCorolla.xlsx", sheet_name="data")
print(corolla.shape)

# Drop any rows with missing values (if any)
corolla = corolla.dropna()
print(corolla.shape)

(1436, 39)
(1436, 39)


In [7]:
# prior to splitting data we perfrom some data preparation steps that propagate through the training, validation, and test data sets
# and save time performing each step on those data sets individually in subsequent tasks

#the first of thes stepse is lmiting the columns of data to those fields listed in question 3

predictors = ["Age_08_04", "KM", "Fuel_Type", "HP", "Automatic", "Doors", "Quarterly_Tax", "Mfr_Guarantee",
"Guarantee_Period", "Airco", "Automatic_airco", "CD_Player", "Powered_Windows",
"Sport_Model", "Tow_Bar"]
outcome = ["Price"]

corrola_data = corolla[predictors + outcome].copy()
corrola_data.head()

,Age_08_04,KM,Fuel_Type,HP,Automatic,Doors,Quarterly_Tax,Mfr_Guarantee,Guarantee_Period,Airco,Automatic_airco,CD_Player,Powered_Windows,Sport_Model,Tow_Bar,Price
0,23,46986,Diesel,90,0,3,210,0,3,0,0,0,1,0,0,13500
1,23,72937,Diesel,90,0,3,210,0,3,1,0,1,0,0,0,13750
2,24,41711,Diesel,90,0,3,210,1,3,0,0,0,0,0,0,13950
3,26,48000,Diesel,90,0,3,210,1,3,0,0,0,0,0,0,14950
4,30,38500,Diesel,90,0,3,210,1,3,1,0,0,1,0,0,13750


In [8]:
#next we convert the column "Fuel_Type" to dummy variables to accomodate data building in task 3 and Part II

corrola_enc = corrola_data.copy()
corrola_enc = pd.get_dummies(corrola_enc, columns=["Fuel_Type"], dtype = "int", drop_first=True)
corrola_enc.head()

,Age_08_04,KM,HP,Automatic,Doors,Quarterly_Tax,Mfr_Guarantee,Guarantee_Period,Airco,Automatic_airco,CD_Player,Powered_Windows,Sport_Model,Tow_Bar,Price,Fuel_Type_Diesel,Fuel_Type_Petrol
0,23,46986,90,0,3,210,0,3,0,0,0,1,0,0,13500,1,0
1,23,72937,90,0,3,210,0,3,1,0,1,0,0,0,13750,1,0
2,24,41711,90,0,3,210,1,3,0,0,0,0,0,0,13950,1,0
3,26,48000,90,0,3,210,1,3,0,0,0,0,0,0,14950,1,0
4,30,38500,90,0,3,210,1,3,1,0,0,1,0,0,13750,1,0


In [9]:
# WIth these data preparation steps complete, we split the data into training (50%), validation (30%), and test (20%) datasets
from sklearn.model_selection import train_test_split

# Split the data into training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(
    corrola_enc.drop('Price', axis=1), corrola_enc['Price'], test_size=0.5, random_state=42)

# Now split the remaining dataset into validation and test datasets.
# 40% of the remaining 50% is 30% of total
X_valid, X_test, y_valid, y_test = train_test_split(
    X_rem, y_rem, test_size=0.4, random_state=42)

# shape is (# of rows, # of columns)
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)
print(X_test.shape)
print(y_test.shape)

(718, 16)
(718,)
(430, 16)
(430,)
(288, 16)
(288,)


### 3.  Run a multiple linear regression with the outcome variable Price and predictor variables: Age_08_04, KM, Fuel_Type, HP, Automatic, Doors, Quarterly_Tax, Mfr_Guarantee, Guarantee_Period, Airco, Automatic_airco, CD_Player, Powered_Windows, Sport_Model, and Tow_Bar.

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

print(lr_model.intercept_)
print(lr_model.coef_)


9742.208352728834
[-1.08704629e+02 -1.77211025e-02  3.25122471e+01  5.77899481e+02
  1.94880208e+02  1.54915042e+01  2.77396627e+02  6.81605818e+01
  2.46572488e+02  2.85415537e+03  3.62275419e+02  4.19159345e+02
  3.96351672e+02 -2.56982042e+02  1.84717700e+03  1.85612566e+03]


### 4.  What appear to be the three or four most important car specifications for predicting the car’s price?

### 5. Using metrics, you consider useful, assess the performance of the model in predicting price.

# Part II


### 1. Data Preprocessing. Split the data into training (60%), and validation (40%) datasets.

### 2. Run a full-grown regression tree (RT) with outcome variable Price and predictors Age_08_04, KM, Fuel_Type (first convert to dummies), HP, Automatic, Doors, Quarterly_Tax, Mfr_Guarantee, Guarantee_Period, Airco, Automatic_airco, CD_Player, Powered_Windows, Sport_Model, and Tow_Bar. Set random_state=1.

### 3. Which appear to be the three or four most important car specifications for predicting the car’s price?

### 4. Compare the prediction errors of the training and validation sets by examining their RMS error and by plotting the two boxplots. How does the predictive performance of the validation set compare to the training set? Why does this occur?

### 5. Predict the price, using the smaller RT of a used Toyota Corolla with the specifications listed in Table 1, see below.